<a href="https://colab.research.google.com/github/svenpete/Adressbuch/blob/master/BvL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Datenaufbereitung einer industriellen Reinigungsanlage**
# Der Niagara DFS             27.7.2018 

Das folgendende Colab-Notebook wird zur Datenaufbereitung von Maschinendaten, einer industriellen Reinigungsanlage verwendet. Bei der industriellen Reinigungsanlge handelt es sich um eine Niagara DFS der Firna BvL Oberflächentechnik. Diese Taktanlage wird für industrielle Komponenten bzw. Bauteile eingesetzt, bei denen eine sehr hohe technischen Sauberkeit maßgebend ist. 

Unter dem Begriff Technische Sauberkeit ist die hinreichend geringe Kontamination sauberkeitssensibler technischer Bauteile mit schädlichen Partikeln zu verstehen. ... die korrekte Funktion des Bauteils bzw. der Baugruppe beeinträchtigen oder verhindern können. (Wikipedia stand 13.07.2021)

Im folgenden werden die Maschinendaten so transformiert & manipuliert, dass diese mit Hilfe von maschinellen Lernverfahren verarbeitet werden können. Zunächst muss dafür eine Bibliothek zur Datenaufbereitung und -analyse gewählt werden. 

Die Datenaufbereitung findet in Python statt, als Bibliothek zur zur Datenaufbereitung und -anlyse wird Pandas verwendet. Das Framework zur Realisierung der maschinellen Lernverfahren ist Tensorflow.


In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

# Was sind diese Maschinendaten?
Bei den Maschinendaten handelt es sich um Sensoren, welche intern und extern an der Reinigungsanlage angebracht wurden. Diese Sensoren zeichnen unteranderem:

Einheit = Bar 
1. Druck der Wasch-, Spülpumpe 1 & 2, Waschbox 
2. Druck vor/nach Filter nach Wasch-, Spülpumpe 1 & 2, Waschbox   
3. Differenzdruck Wasch-, Spülpumpe 1 & 2, Waschbox 
4. Kammerdruck

Einheit = Celsius
5. Temperatur der Vakuumtrockner 1 & 2 
6. Temperatur von Waschtank, Spültank 1 & 2 
7. Temperatur der Waschbox

Einheit = Zustandslos (Libellen für  Wasch-, Spülpumpe 1 & 2)
6. Verschmutzung des Wassers = Libelle Fluid
7. Verschmutzung des Wassers mit Öl = Libelle Oil
8. Reinigeranteil im Wassser = Libelle Cleaner 

Einheit = Sonstige
9. Taktzeit
10. Werkstückzähler 
11. Maschinenzustand

Einheit = S ( Simens)
12. Leitwert Wasch-, Spülpumpe 1 & 2, Waschbox 


Insgesamt werden 23 verschiedene Sensoriken erfasst. Einige Sensoren zeichnen während des gesamten Zeitraums eines Waschvorgangs Daten auf. Wiederrum andere Sensoren, zeichnen Daten nur während eines Teilprozesses vom Waschgang auf z.B. der Druck der Spülpumpe 1 vor dem Filter tritt nur während des Teilprozesses "Spülen 1" auf.



# Erfassung der Maschinendaten
Die Maschinendaten werden von einem Datalogger erfasst. Dieser erfasst eine Änderungen wenn der vorherige Wert sich um 5% geändert hat oder 5 Minuten vergangen sind. Die Daten werden als CSV-Format gespeichert. Es wird für jeden Monat eine neue CSV angelegt. 


# Problem Datalogger
Durch das spezifizierte Verhalten des Dataloggers,kann die Anzahl an Sensormesswerten für verschiedene Waschgänge stark variieren.
Zu einem ist es wichtig für jeden Waschgang eine einheitliche Anzahl an Sensormesswerten zu haben. 
Für die Maschinendaten eine Datei mit allen Sensormesswerten für den gesamten Zeitraums anzulegen. (Header der CSV ist 4 Zeilen groß) Hierdurch lassen sich statischtische Auswertungen und Visualisierungen später einfacher und schneller anwenden.


# Laden der Maschinendaten

Zunächst werden die Maschinendaten in ein Datenframe geladen. Das Datenframe aus der Python Bibliothek "Pandas" bietet umfangreiche Funktionalitäten zur Manipulation und Transformierung von Daten was es sehr Mächtig macht und deshalb zur Datenaufbereitung verwendet wird.
Anschließend werden Metadaten des Datenframes abgerufen geladen über den:

1. Shape = Drückt Zeilen und Spaltencount des Datenframes aus
2. Head = Gibt die ersten 5 Zeilen der Datenframes zurück
3. Columms = Namen aller Spalten
4. dtype = Datentypen der einzlenen Columns

In [ ]:
# Get the libary
import pandas as pd
# loading data from google drive 
df_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Testing.csv', parse_dates=['Timestring'],sep=";",low_memory=False, header=0, cache_dates=True)
print(df_data.shape)
print(df_data.columns)
print(df_data.dtypes)
print(df_data.head())

(8274229, 10)
Index(['New_ID', 'Index', 'Var_Name', 'Timestring', 'Seconds', 'Milliseconds',
       'Value', 'Day of Week', 'Sensorname', 'Date2'],
      dtype='object')
New_ID                   int64
Index                  float64
Var_Name                object
Timestring      datetime64[ns]
Seconds                float64
Milliseconds           float64
Value                   object
Day of Week             object
Sensorname              object
Date2           datetime64[ns]
dtype: object
   New_ID  Index  ...                Sensorname      Date2
0       0    0.0  ...  Temperatur Vakuumpumpe 2 2020-02-05
1       1    1.0  ...       Leitwert Spültank 2 2020-02-05
2       2    2.0  ...       Leitwert Spültank 1 2020-02-05
3       3    3.0  ...  Temperatur Vakuumpumpe 1 2020-02-05
4       4    4.0  ...        Leitwert Waschtank 2020-02-05

[5 rows x 10 columns]


# Laden der Waschergebnisse
 Als nächstes werden die Waschergebnisse geladen. Diese werden regelmäßig Stichprobenartig entnommen und im Labor untersucht. Dabei wird das Waschgut in einem Labor, auf die technische Sauberkeit überprüft (s.O.). Es wird zwischen i.O. und n.i.O. unterschieden. Dafür  ausschlaggebened sind die Partikel auf der Öberfläche nach einem Waschgang. 

 1. Das Waschergebnis ist i.O. wenn alle Partikel < 600 nm
 2. Das Waschergebnis ist n.i.O. wenn ein Partikel > 600 nm

Die Waschergebnisse werden für den gesamten Tag betrachtet. Dadurch kann die KI später mehrere hunderte Cyclen als nur ein paar wenige. 

Neben dem Datum und dem Waschergebniss wird auch noch das gewaschenen Bauteil gespeichert. Auf Grund der Ähnlichkeit der Bauteile und Stichproben wird immer vom gleichen Bauteil ausgegangen.

In [ ]:
df_results = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Ratings.csv',parse_dates=['Date'], sep=";",low_memory=False, header=0, cache_dates=True)
print(df_results.shape)
print(df_results.head())
print(df_results.dtypes)

(22, 3)
        Date     Component  Rating
0 2020-09-15    Hauptwelle   i. O.
1 2020-09-23    Hauptwelle   i. O.
2 2020-09-29    Hauptwelle   i. O.
3 2020-10-01  Hauptwelle 2   i. O.
4 2020-10-01  Hauptwelle 1  i. O. 
Date         datetime64[ns]
Component            object
Rating               object
dtype: object


# Zusammenfügen der Maschinendaten und Waschergebnissen
Im diesem Abschnitt werden nun die Maschinendaten mit den Waschergebnissen zusammengefügt. 
1. Da die Maschinendaten nur einen Timestamp besitzen, wird eine zusätzliche Spalte mit einem Datum hinzugefügt.
2. Danach wird mit dem Datum der Maschinendaten ein "Left Join" mit dem Datum der Waschergebnissen durchgeführt.
3. Die Maschinendaten enthalten nun Waschergebnissen für 

In [ ]:
# change dtypes for merge
df_results['Date'] = pd.to_datetime(df_results['Date'])

# add column to machine data and change dtype for merge otherwise no result
df_data['Date2'] = df_data['Timestring'].dt.date
df_data['Timestring'] = pd.to_datetime(df_data['Timestring'])
df_data['Date2'] = pd.to_datetime(df_data['Date2'])c

# get shapes
print(df_results.shape)
print(df_data.shape)

# Left Join is needed
df = df_data.merge(df_results, left_on='Date2', right_on='Date', how='left', sort='False')

# delete rows from dataframe to store
df.drop(columns=['New_ID', 'Index', 'Seconds', 'Milliseconds', 'Day of Week', 'Date2', 'Sensorname'], inplace=True, axis=1)

# get first columns and store results in onedrive  
print(df.head(5))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/Merged_Data.csv', sep=';', index=False)


(22, 3)
(8274229, 10)
New_ID                0
Index                 0
Var_Name              0
Timestring            0
Seconds               0
Milliseconds          0
Value                 0
Day of Week           0
Sensorname            0
Date2                 0
Date            7750835
Component       7750835
Rating          7750835
dtype: int64
(8301246, 13)
          Var_Name          Timestring Value Date Component Rating
0      temp_vp2_vc 2020-02-05 11:34:44    54  NaT       NaN    NaN
1  med_cond_rt2_wc 2020-02-05 11:35:20  6016  NaT       NaN    NaN
2   med_cond_rt_wc 2020-02-05 11:35:22  8043  NaT       NaN    NaN
3       temp_vp_wc 2020-02-05 11:35:29    55  NaT       NaN    NaN
4   med_cond_wt_wc 2020-02-05 11:35:41  8983  NaT       NaN    NaN


# Transformieren der Maschinendaten
Die Maschinendaten werden vom Datalogger tabellarisch und chronologisch untereinander gespeichert. Was beim Mergen bereits zu sehen war. Damit die Sensordaten als Features für maschinelle Lernverfahren verwendet werden kann, sollte die Sensornamen als Spalten transformiert werden.

Jeder Sensor erhält eine Spalte mit seinem Namen. In dieser Spalte wird der Wert für den jeweiligen Zeitpunkt chronologisch gespeichert.

In [1]:
import numpy as np
import pandas as pd 
# get data and check types
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Merged_Data.csv',parse_dates=['Timestring'],sep=";",low_memory=False, header=0, cache_dates=True)

#get  unique sensornames
name_arr = df['Var_Name'].unique()


df_transformed_machinedata = pd.DataFrame()

for name in name_arr:

  # get dataframe for each sensor 
  df_tmp = df.loc[df['Var_Name'] == name, ['Timestring', 'Value']]
  df_tmp.set_index('Timestring', inplace=True)

  # rename label from value to sensorname
  df_tmp.rename(columns = {'Value' : name}, inplace=True)
 
  # concat dataframe piece horizontal
  df_transformed_machinedata = pd.concat([df_transformed_machinedata,df_tmp],)
  
#indexing for sorting transformend machinedata 
df_transformed_machinedata.reset_index(inplace=True)
df_transformed_machinedata.set_index('Timestring', inplace=True)
df_transformed_machinedata.sort_index(inplace=True)
df_transformed_machinedata.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/Sensordata_ordered.csv', sep=';', index=True)

KeyboardInterrupt: ignored

# Problem der Prozesszeiten von Waschgänge 
Die Ermittlung der Prozesszeiten von Waschgänge und die daraus hervorgehende Strukturierung der Daten ist sehr schwierig. 

Die Prozesszeiten der Waschgänge varriert, dies kann mit Fehler beim Ab- oder Antransport eines Waschgutes entstehen. Oder am HMI der Reinigungsanlage wurden die Prozesszeiten umgestellt. 
Die Prozesszeiten sowie änderungen daran werden nicht gespeichert.

-> Die Prozesszeit für ein Waschgang kann jedoch anhand des Werkstückzählers und der Waschpumpe bestimmt.

# Ermittlung der dynamischen Prozesszeiten von Waschgänge 
Im folgenden wird das Vorgehen zur Bestimmung der dynamischen Prozesszeiten für Waschgänge erläutert. Der Sensoriken:

1.Werkstückzähler (odata_count_wc) = Dient als inkrementeller Zähler bis dato der gereinigten Waschgüter. Wird am Ende jedes Waschgangs erst erhöht.

Aussagen Werkstückzähler
Max - Min = Anzahl aller Waschgänge
Numerische Nummerierung ab 1 = Aktuellen Waschgang 



2. Druck vor Filter Waschpumpe (pres_wp_fi_wc) = Der Waschgang beginnt immer zuerst mit dem Teilprozess Waschen. Der Sensor für den Druck vor dem Filter der Waschpumpe sollte ansteigen und der Datenlogger speichert den Wert ab, wegen der 5% Regelung von Wertänderungen. Dies hat zur Folge, dass bei Start des Teilprozesses Waschen immer Wert von 0,1 Bar gespeichert wird. Es kann vorkommen, dass dieser Wert in einigen Szenarien auch 0,2 Bar beträgt. Deshalb wird 0,2 als Obergrenze für den Start des Waschgangs.

Formel
x <= 0.2 = Startprozess 

# Problematik zu 2
Die Waschpumpe wird im Teilprozess Waschen als erstes vom Düsenrahmen verwendet und 2-3 Minuten später dann nochmals fürs Fluten der Waschkammer. Hier würde die Logik aus dem Absatz davor nicht greifen, weil die Waschpumpe zu einer falschen Aussage führen würde.


# Lösugen mit delta Timestamp

# Endwert 
Die Zeitdifference zwischen den Werkstückzählern und den Druck vor Filter Waschpumpe kann zur Validierung von Datenpunkten verwendet werden. Zu lange und zu kurze Zeitintervalle werden erstmals nicht berücksichtigt mit dieser Methode.

Als Ober- und Untergrenze für den Werkstückzähler wird verwendet:

Erwarungswert - Varianz = Untergrenze
Erwarungswert + Varianz = Obergrenze ( noch keine Obergrenze Verhalten noch erforschen)
 
Durchschnittlicher Waschgang beträgt ca. 24-25 Min.
(Auftreten von odata_count_wc)

#Startwert 
Für den Startzeitpunkt des Waschgangs sollte eine Obergrenze für den zeitlichen Abstand dieser Sensorwerte gesetzt werden. 
Die Obergrenze für das Timedelta sollte der Abstand zum letzten Sensorwert des letzten Waschgangs von "Druck vor Filter Waschpumpe" sein.

1. Durchschnittlich dauert die Düsenreinigung ca. 2.40 - 3.20 Minuten
2. Durchschnittlich dauert die Düsenreinigung ca. 3.10 Minuten
3. Durchschnittlich dauert das Fluten der Waschkammer ca. 1.10 Minuten

Insgesamt = 7,40 Minuten Waschen

-> Die Obergrenze sollte niemals kleiner sein als die Durchschnittliche Zeit der aufgeführten 3 Punkte. Der Wert für die Obergrenze der Deltatime der Waschpumpe sollte nicht größer als oder gleich der Obergrenze für den Werkstückzähler sein. Der Wert sollte zwischen [12.50 - 21.30] liegen. 


In [1]:
import pandas as pd
import numpy as np
import datetime

#read data and use specified columms
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Sensordata_ordered.csv', usecols=['Timestring', 'odata_count_wc', 'pres_wp_fi_wc'], parse_dates=['Timestring'],sep=";",low_memory=False, header=0, cache_dates=True)

#sort data if not sorted 
df.set_index('Timestring', inplace=True)
df.sort_index()

#get value and nan value count
print(df.count())
print(df.odata_count_wc.isnull().sum())

#delete data which has more than 1 nan in its column
df.dropna(thresh=1, inplace=True)

#replace , with . so you can these values as floats
df['pres_wp_fi_wc'] = df['pres_wp_fi_wc'].str.replace(',', '.').astype(float)

#apply used to cast objects to numeric values in dataframe 
df.apply(pd.to_numeric)

pres_wp_fi_wc     372955
odata_count_wc     10408
dtype: int64
8290838


,pres_wp_fi_wc,odata_count_wc
Timestring,,
2020-02-05 11:35:59,0.1,NaN
2020-02-05 11:36:01,2.0,NaN
2020-02-05 11:36:01,3.8,NaN
2020-02-05 11:36:02,9.4,NaN
2020-02-05 11:36:02,7.4,NaN
...,...,...
2021-05-26 08:00:49,6.8,NaN
2021-05-26 08:00:49,5.0,NaN
2021-05-26 08:00:49,9.8,NaN


# Bestimmung des Startzeitpunktes vom Waschgang
Brechnung der Deltatime für Durck vor Filter Waschpumpe


In [2]:
from datetime import timedelta

#press indicates the start of a process 
press_value_series = df['pres_wp_fi_wc']

#drop empty values | inplace = store changes to dataframe
press_value_series.dropna(inplace=True)

#generate df for delta calc
df_press = pd.DataFrame(press_value_series)

#store timestamp for accessing as value 
df_press['delta'] = df_press.index

# shifts the value delta calc.
df_press['delta_time'] = df_press.delta.diff()
df_press.dropna(inplace=True)

#get delta_time_series for vald.
delta_time_value_series = df_press['delta_time']

print(df_press.describe())
print(df_press)

       pres_wp_fi_wc                 delta_time
count  372954.000000                     372954
mean        6.350054  0 days 00:01:50.237452876
std         4.279299  0 days 02:07:22.810508291
min         0.100000            0 days 00:00:00
25%         2.500000            0 days 00:00:00
50%         6.400000            0 days 00:00:01
75%         9.400000            0 days 00:00:08
max        16.200000           44 days 10:17:48
                     pres_wp_fi_wc               delta      delta_time
Timestring                                                            
2020-02-05 11:36:01            2.0 2020-02-05 11:36:01 0 days 00:00:02
2020-02-05 11:36:01            3.8 2020-02-05 11:36:01 0 days 00:00:00
2020-02-05 11:36:02            9.4 2020-02-05 11:36:02 0 days 00:00:01
2020-02-05 11:36:02            7.4 2020-02-05 11:36:02 0 days 00:00:00
2020-02-05 11:36:02            5.6 2020-02-05 11:36:02 0 days 00:00:00
...                            ...                 ...             ...


#Bestimmung des Endzeitpunktes vom Waschgang
Brechnung der Deltatime für den Werkstückzähler

In [3]:
#to determine ende of washprocess cycle
odata_series = df['odata_count_wc']

#drop empty values
odata_series = odata_series.dropna()

#generate df for delta calc
df_odata_count  = pd.DataFrame(odata_series)

#store timestamp for accessing as value 
df_odata_count['delta'] = df_odata_count.index

# shifts the value delta calc.
df_odata_count['delta_time'] = df_odata_count.delta.diff()

print(df_odata_count.describe())
print(df_odata_count)

       odata_count_wc                 delta_time
count    10408.000000                      10407
mean     20130.770177  0 days 01:05:50.143172864
std       3139.647660  0 days 12:49:19.722942746
min      14846.000000            0 days 00:00:00
25%      17446.750000            0 days 00:22:49
50%      20008.500000            0 days 00:23:43
75%      22784.250000            0 days 00:24:38
max      26067.000000           45 days 04:41:20
                     odata_count_wc               delta      delta_time
Timestring                                                             
2020-02-05 12:37:03         14846.0 2020-02-05 12:37:03             NaT
2020-02-05 12:40:12         14847.0 2020-02-05 12:40:12 0 days 00:03:09
2020-02-05 13:03:47         14848.0 2020-02-05 13:03:47 0 days 00:23:35
2020-02-05 13:27:37         14849.0 2020-02-05 13:27:37 0 days 00:23:50
2020-02-05 13:53:16         14850.0 2020-02-05 13:53:16 0 days 00:25:39
...                             ...                 ...

# Ermittlung der Waschgangsintervalle
Bestimmung von Waschgangsintervalle bei industriellen Reinigungsanlagen anhand von Sensoriken. Zudem Erstellung und Speicherung des neuen Datenframes.


# ***Es sollte geprüft werden ob ein Washzylkus Daten über den Anstieg der Pumpe enthält

In [4]:
import datetime


#timedelta for comparison
cap = timedelta(minutes=10, seconds=1)
cap_odata = timedelta(minutes=17, seconds=1)


#filter timedeltas from start and end 
start_cycle = df_press.loc[(df_press['delta_time'] > cap ) & (df_press['pres_wp_fi_wc'] <= 0.2)].index.to_list()
end_cycle = df_odata_count.loc[df_odata_count['delta_time'] > cap_odata].index.to_list()


df_cycle_data = []
cycle_number = 0 # counts washcycles
data_counter = 0  # count data entry in washcycle

In [5]:
print(start_cycle)

[Timestamp('2020-02-05 12:17:40'), Timestamp('2020-02-05 12:44:40'), Timestamp('2020-02-05 13:08:30'), Timestamp('2020-02-05 13:34:06'), Timestamp('2020-02-05 13:57:54'), Timestamp('2020-02-05 14:21:48'), Timestamp('2020-02-05 15:55:52'), Timestamp('2020-02-05 16:19:44'), Timestamp('2020-02-05 16:52:02'), Timestamp('2020-02-05 17:15:52'), Timestamp('2020-02-05 17:39:46'), Timestamp('2020-02-05 18:08:16'), Timestamp('2020-02-05 18:32:09'), Timestamp('2020-02-05 18:55:55'), Timestamp('2020-02-05 19:19:49'), Timestamp('2020-02-05 19:43:35'), Timestamp('2020-02-05 20:07:28'), Timestamp('2020-02-05 20:31:14'), Timestamp('2020-02-05 21:20:37'), Timestamp('2020-02-05 21:44:24'), Timestamp('2020-02-05 22:08:18'), Timestamp('2020-02-05 22:32:04'), Timestamp('2020-02-05 22:56:04'), Timestamp('2020-02-05 23:21:11'), Timestamp('2020-02-05 23:45:07'), Timestamp('2020-02-06 00:08:53'), Timestamp('2020-02-06 00:32:53'), Timestamp('2020-02-06 00:56:39'), Timestamp('2020-02-06 01:20:38'), Timestamp('20

In [4]:
import datetime


#timedelta for comparison
cap = timedelta(minutes=10, seconds=1)
cap_odata = timedelta(minutes=17, seconds=1)


#filter timedeltas from start and end 
start_cycle = df_press.loc[(df_press['delta_time'] > cap ) & (df_press['pres_wp_fi_wc'] <= 0.2)].index.to_list()
end_cycle = df_odata_count.loc[df_odata_count['delta_time'] > cap_odata].index.to_list()


df_cycle_data = []
cycle_number = 0 # counts washcycles
data_counter = 0  # count data entry in washcycle

# iterate over odata_count_wc timestrings 
for end_date in end_cycle:
  

  # iterate over washpump timestrings
  for start_date in start_cycle:
   
    
    if data_counter == 0:
      
      #store information for dataframe 
      df_cycle_data.append([str(start_date), str(end_date), str(cycle_number)])
      data_counter = data_counter + 1    
      continue
    
    #check if bigger than remove and get new endindex
    if start_date < end_date:
      data_counter = data_counter + 1
      continue

    break
  
  #delete used dates for performance obvious
  del start_cycle[:data_counter]
  cycle_number = cycle_number + 1 
  data_counter = 0  
  
#store results to csv
df_cycle = pd.DataFrame(df_cycle_data, columns =['Start', 'End', 'Cycle'])
#df_cycle.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/Waschzyklen.csv', index=False, sep=';')

In [ ]:
df_cycle

# Visualisierung und Analyse der Waschzyklen 
Der folgende Abschnitt enthält VIsualisierungen zur Veranschaulichung der Waschzyklen so wie einige statische Merkmale.

# Maschinendaten anhand der Waschzyklen strukturieren
Im folgenden werden die Maschinendaten mit den Waschzyklen zusammengeführt. Die Verknpüfung wird durch einen Left Join auf den Maschinendaten realisiert. Hierfür werden synthetische Schlüssel angelegt, die später verworfen werden.

In [2]:
import pandas as pd 
import numpy as np
import seaborn
import matplotlib.pyplot as plt


df_washcycles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Waschzyklen.csv', parse_dates=['Start', 'End'],sep=";",low_memory=False, header=0, cache_dates=True)
print(df_washcycles.head())


df_sensordata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Sensordata_ordered.csv', parse_dates=['Timestring'],sep=";",low_memory=False, header=0, cache_dates=True)
washcycle_dates = list(df_washcycles.itertuples(index=False,name=None))
df_sensordata.set_index('Timestring', inplace=True)
df_sensordata['cycle'] = 0
df_sensordata.reset_index(inplace=True)
df_sensordata['index'] = df_sensordata.index
df_sensordata.head()



                Start                 End  Cycle
0 2020-02-05 12:17:40 2020-02-05 13:03:47      0
1 2020-02-05 13:08:30 2020-02-05 13:27:37      1
2 2020-02-05 13:34:06 2020-02-05 13:53:16      2
3 2020-02-05 13:57:54 2020-02-05 14:17:05      3
4 2020-02-05 14:21:48 2020-02-05 14:43:38      4


,Timestring,temp_vp2_vc,med_cond_rt2_wc,med_cond_rt_wc,temp_vp_wc,med_cond_wt_wc,odata_osta_wc,pres_wp_fi_wc,pres_wp_fo_wc,pres_air_wc,pres_wp_diff_wc,temp_rt_wc,temp_wt_wc,temp_wp_at,pres_wp_fi_at,pres_wp_fo_at,pres_wp_diff_at,pres_rp_fi_wc,pres_rp_fo_wc,pres_rp_diff_wc,pres_rp2_fi_wc,pres_rp2_fo_wc,pres_rp2_diff_wc,odata_count_wc,odata_cyti_wc,med_lo_wt_wc,med_lo_rt_wc,med_lo_rt2_wc,med_lc_wt_wc,med_lc_rt_wc,med_lc_rt2_wc,med_lf_wt_wc,med_lf_rt_wc,med_lf_rt2_wc,temp_rt2_wc,cycle
0,2020-02-05 11:34:44,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2020-02-05 11:35:20,NaN,6016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2020-02-05 11:35:22,NaN,NaN,8043.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2020-02-05 11:35:29,NaN,NaN,NaN,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2020-02-05 11:35:41,NaN,NaN,NaN,NaN,8983.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
# df_sensordata.reset_index(inplace=True)
# df_sensordata.set_index('Timestring', inplace=True)
# df_sensordata['Timestring'] = df_sensordata.index
df_sensordata.head()

,Timestring,temp_vp2_vc,med_cond_rt2_wc,med_cond_rt_wc,temp_vp_wc,med_cond_wt_wc,odata_osta_wc,pres_wp_fi_wc,pres_wp_fo_wc,pres_air_wc,pres_wp_diff_wc,temp_rt_wc,temp_wt_wc,temp_wp_at,pres_wp_fi_at,pres_wp_fo_at,pres_wp_diff_at,pres_rp_fi_wc,pres_rp_fo_wc,pres_rp_diff_wc,pres_rp2_fi_wc,pres_rp2_fo_wc,pres_rp2_diff_wc,odata_count_wc,odata_cyti_wc,med_lo_wt_wc,med_lo_rt_wc,med_lo_rt2_wc,med_lc_wt_wc,med_lc_rt_wc,med_lc_rt2_wc,med_lf_wt_wc,med_lf_rt_wc,med_lf_rt2_wc,temp_rt2_wc,cycle,index
0,2020-02-05 11:34:44,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,2020-02-05 11:35:20,NaN,6016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2,2020-02-05 11:35:22,NaN,NaN,8043.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
3,2020-02-05 11:35:29,NaN,NaN,NaN,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
4,2020-02-05 11:35:41,NaN,NaN,NaN,NaN,8983.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4


In [6]:
df_washcycles['start_id'] = 0
df_washcycles['end_id'] = 0
df_washcycles

,Start,End,Cycle,start_id,end_id
0,2020-02-05 12:17:40,2020-02-05 13:03:47,0,0,0
1,2020-02-05 13:08:30,2020-02-05 13:27:37,1,0,0
2,2020-02-05 13:34:06,2020-02-05 13:53:16,2,0,0
3,2020-02-05 13:57:54,2020-02-05 14:17:05,3,0,0
4,2020-02-05 14:21:48,2020-02-05 14:43:38,4,0,0
...,...,...,...,...,...
10331,2021-05-26 04:23:12,2021-05-26 04:42:20,10331,0,0
10332,2021-05-26 04:45:56,2021-05-26 05:04:57,10332,0,0
10333,2021-05-26 06:43:58,2021-05-26 07:03:04,10333,0,0
10334,2021-05-26 07:06:40,2021-05-26 07:25:40,10334,0,0


In [9]:
for value in washcycle_dates:

  min = value[0]
  max = value[1]
  iter = value[2]

  print('Iteration: ' + str(iter))

  df_tmp = df_sensordata.query('@min <= Timestring <= @max')
  min_id = df_tmp.index.min()
  max_id = df_tmp.index.max()

  print(min_id)
  print(max_id)

  try:
    df_washcycles.at[value[2],'start_id'] = min_id
    df_washcycles.at[value[2],'end_id'] = max_id
    break
  except ValueError:
    print("Error ------ Start")
    print(min)
    print(min)
    print(min_id)
    print(max_id)
    print("Error ------ End")
    break
    df_washcycles.at[value[2],'start_id'] = -1
    df_washcycles.at[value[2],'end_id'] = -1


Iteration: 0
232
1840


In [10]:
df_washcycles

,Start,End,Cycle,start_id,end_id
0,2020-02-05 12:17:40,2020-02-05 13:03:47,0,232,1840
1,2020-02-05 13:08:30,2020-02-05 13:27:37,1,1863,2400
2,2020-02-05 13:34:06,2020-02-05 13:53:16,2,2432,2993
3,2020-02-05 13:57:54,2020-02-05 14:17:05,3,3017,3924
4,2020-02-05 14:21:48,2020-02-05 14:43:38,4,3997,4712
...,...,...,...,...,...
10331,2021-05-26 04:23:12,2021-05-26 04:42:20,10331,0,0
10332,2021-05-26 04:45:56,2021-05-26 05:04:57,10332,0,0
10333,2021-05-26 06:43:58,2021-05-26 07:03:04,10333,0,0
10334,2021-05-26 07:06:40,2021-05-26 07:25:40,10334,0,0


In [8]:
for value in washcycle_dates:

  min = value[0]
  max = value[1]
  df_tmp = df_sensordata.query('@min <= Timestring <= @max')
  try:
    df_washcycles.set_value('start_id', value[2],df_tmp.index.min())
    df_washcycles.set_value('end_id', value[2],df_tmp.index.max())
    # df_washcycles.at[value[2],'start_id'] = df_tmp.index.min()
    # df_washcycles.at[value[2],'end_id'] = df_tmp.index.max()
  except ValueError:
    df_washcycles.set_value('start_id', value[2],df_tmp.index.min())
    df_washcycles.set_value('end_id', value[2],df_tmp.index.max())

    # df_washcycles.at[value[2],'start_id'] = -1
    # df_washcycles.at[value[2],'end_id'] = -1


AttributeError: ignored

In [3]:
for value in washcycle_dates:

  min = value[0]
  max = value[1]

  df_tmp = df_sensordata.query('@min <= Timestring <= @max')
  df_washcycles.loc[value[2]] = df_tmp.index.min()
  df_washcycles.loc[value[2]] = df_tmp.index.max()
  print(df_tmp.index.max())
  #cycle = df_sensordata.loc[value[0] : value[1]]
  #cycle['cycle'] = value[2]
  #filter = (df_sensordata['Timestring'] >= min) & (df_sensordata['Timestring'] <= max), 'cycle'
  #df_sensordata.loc[filter] = value[2]


  #print(df_sensordata.loc[(df_sensordata['Timestring'] >= min) & (df_sensordata['Timestring'] <= max), 'cycle'] = value[2])


  

  #filter = (df_sensordata['Timestring'] >= value[0]) & (df_sensordata['Timestring'] < value[1])
  #index_to_update = df_sensordata['index'].[filter]
  #df_sensordata.apply(df_sensordata.query('@min <= Timestring <= @max').cycle = @value[2])
  #print(df_sensordata.query('@min <= Timestring <= @max').cycle)
  
  #df_sensordata. = df.apply(lambda r: df.query('@r.Date_Time <= index <= @r.d1_idx_last').Low.min(), axis=1)

  #df_sensordata.between_time(value[0], value[1])
 
    #[index_to_update[i], 'cycle'] = str(value[2])
  #print(df_sensordata['cycle'].count)
  
  #df_sensordata.loc[min : max,'cycle'] = str(value[2])
  #print(df_tmp)
  #df_tmp.
  # print(df_sensordata)
  # print(df_sensordata.loc[index_to_update.min() : index_to_update.max(),['cycle']])
  # # cycle.to_csv('/content/drive/MyDrive/Colab Notebooks/3184/Cycle Data/cycle_' +
  #              str(value[2]) + '_from_' + str(value[0]) + '_to_' + str(value[1])
  #              + '_M3184.csv', sep=';', index=True)

# df_sensordata.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/Sensordata_with_cycle.csv', sep=';', index=False)


232
1840
1863
2400
2432
2993
3017
3924
3997
4712
5159
6025
6050
6757
6813
7413
7438
8114
8139
8674
8716
9321
9346
9865
9890
10445
10471
11092
11114
11786
11810
12365
12390
12939
13065
13634
13658
14354
14380
14969
14993
15708
15733
16378
16407
17000
17026
17636
17659
18317
18342
19050
19075
19756
19782
20335
20361
20996
21022
21620
21643
22273


KeyboardInterrupt: ignored

In [ ]:
df_sensordata

,Timestring,temp_vp2_vc,med_cond_rt2_wc,med_cond_rt_wc,temp_vp_wc,med_cond_wt_wc,odata_osta_wc,pres_wp_fi_wc,pres_wp_fo_wc,pres_air_wc,pres_wp_diff_wc,temp_rt_wc,temp_wt_wc,temp_wp_at,pres_wp_fi_at,pres_wp_fo_at,pres_wp_diff_at,pres_rp_fi_wc,pres_rp_fo_wc,pres_rp_diff_wc,pres_rp2_fi_wc,pres_rp2_fo_wc,pres_rp2_diff_wc,odata_count_wc,odata_cyti_wc,med_lo_wt_wc,med_lo_rt_wc,med_lo_rt2_wc,med_lc_wt_wc,med_lc_rt_wc,med_lc_rt2_wc,med_lf_wt_wc,med_lf_rt_wc,med_lf_rt2_wc,temp_rt2_wc,cycle
0,2020-02-05 11:34:44,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2020-02-05 11:35:20,NaN,6016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2020-02-05 11:35:22,NaN,NaN,8043.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2020-02-05 11:35:29,NaN,NaN,NaN,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2020-02-05 11:35:41,NaN,NaN,NaN,NaN,8983.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8301241,2021-05-26 08:06:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8768.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8301242,2021-05-26 08:06:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"6,3",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8301243,2021-05-26 08:06:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,3",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8301244,2021-05-26 08:06:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4774.0,NaN,0


In [ ]:
df_sensordata.cycle > 0

0          True
1          True
2          True
3          True
4          True
           ... 
8301241    True
8301242    True
8301243    True
8301244    True
8301245    True
Name: cycle, Length: 8301246, dtype: bool

In [ ]:
#df_sensordata.set_index('Timestring', inplace=True)
# df_sensordata['2021']



df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Sensordata_with_cycle.csv', sep=';', low_memory=False, header=0, cache_dates=True)
df['Timestring'] = pd.to_datetime(df['Timestring'])
df.set_index('Timestring', inplace=True)

In [ ]:
df['2021']

In [ ]:
df_washcycles['delta_time'] = df_washcycles['End'] - df_washcycles['Start']
print(df_washcycles.head())
df_washcycles.describe()
df_sensordata['Index'] = lambda x: for x in range(len(df_sensordata.index))

In [ ]:
index = pd.IntervalIndex.from_tuples(tuples, dtype='interval')

df_interval = pd.DataFrame(df_washcycles,             
                            index = pd.IntervalIndex.from_tuples(tuples, dtype='interval'))




index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])

# change dtypes for merge
df['Date'] = pd.to_datetime(df['Date'])

# add column to machine data and change dtype for merge otherwise no result
df_data['Date2'] = df_data['Timestring'].dt.date
df_data['Timestring'] = pd.to_datetime(df_data['Timestring'])
df_data['Date2'] = pd.to_datetime(df_data['Date2'])

# print(test.loc[test['Start'] == True])
test['cycle_end'] = ((test['End'] != test['End'].shift(1)).cumsum()) 
test['cycle_start'] = ((test['Start'] != test['Start'].shift(1)).cumsum()) 
test['cycle_match'] = ((test['Start'] == test['End'].shift(1)).cumsum()) 
test

Visualisierung

In [ ]:
df_press.boxplot(column=['delta_time'], by=['delta'])

import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt

n = 480
ts = pd.Series(np.random.randn(n), index=pd.date_range(start="2020-02-05", periods=n, freq="H"))


fig, ax = plt.subplots(figsize=(12,5))
seaborn.boxplot(ts.index.dayofyear, ts, ax=ax)

In [ ]:
test = pd.Period()

In [ ]:
print(press_value_series)

Timestring
2020-02-05 11:36:01     2.0
2020-02-05 11:36:01     3.8
2020-02-05 11:36:02     7.4
2020-02-05 11:36:02     9.4
2020-02-05 11:36:02    12.0
                       ... 
2020-02-11 14:26:38     7.9
2020-02-11 14:26:39    14.0
2020-02-11 14:26:44    11.6
2020-02-11 14:26:44     8.7
2020-02-11 14:27:04     9.1
Name: pres_wp_fi_wc, Length: 7857, dtype: float64


Auswahl des ersten Wert als erster Wert der geordeneten Sensordaten

In [ ]:

df_press.delta_time.resample('H').plot()
#resample('15Min').mean()



In [ ]:
from matplotlib import pyplot


#testing.isnull().sum()
df['temp_wt_wc'] = pd.to_numeric(df['temp_wt_wc'])
print(df.shape)
print(df.resample('15Min').mean())



#testing['temp_wt_wc'].plot(style='k')

print(df.shape)

In [ ]:
print(df.shape)
testing.odata_count_wc.plot()

(8301246, 2)


TypeError: ignored

In [ ]:
df.isnull().sum()
print(df.shape)
print(df.dropna(how='any').shape)

In [ ]:
df_results['Date'] = pd.to_datetime(df_results['Date'])
df_data['Date2'] = df_data['Timestring'].dt.date
df_data['Date2'] = pd.to_datetime(df_data['Date2'])
print(df_results.shape)
print(df_data.shape)
df = df_data.merge(df_results, left_on='Date2', right_on='Date', how='left', sort='False')

df.drop(columns=['New_ID', 'Index', 'Seconds', 'Milliseconds', 'Day of Week', 'Date2', 'Date', 'Sensorname'], inplace=True, axis=1)
print(df.shape)



NameError: ignored

In [ ]:
df.isnull().sum()
print(df.shape)
#print(df.dropna(how='any').shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
x[ : ,0] = label_encoder.fit_transform(x[ :, 0])
x

Mergen der Maschinendaten + Waschergebnisse mit einem Dummy


In [ ]:
#df_t.set_index(['Timestring', 'Var_Name'], inplace=True)
dummy = pd.get_dummies(df['Var_Name'])
print(dummy)
dataset = pd.concat([df, dummy], axis=1)
print(dataset)
#dataset.drop( columns=['Var_Name', 'pres_'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
x = onehotencoder.fit_transform(x)

In [ ]:
import numpy as np

for value in end_cycle:
  print(value)
  text = str(value)
  np.datetime64('2002-06-28T01:00:00.000000000+0100').astype(datetime)
  
def parse_dates(date):
  date_format = "%Y-%m-%d %H:%M:%S"
  d_value = value // 10**9
  date2 = int(date)
  print(date2)
  p_date = datetime.datetime.fromtimestamp(date2).strftime('%Y-%m-%d %H:%M:%S')
  parsed_dates.append(p_date)

  return parsed_date


# x = [0, 1, 2]
# y = ''.join(map(str, end_cycle))
# z = int(y)


# dates = parse_dates(end_cycle[1])